# The Newsvendor Model - Section 2: Mathematical Analysis

In Section 1: Numerical Studies, we used numerical simulation to explore the basic ideas of the Newsvendor Problem -- how your order quantity decision in the face of random demand impacts your sales, revenue, cost and profit for the day.

Here, we will repeat the same exploration but using mathematical analysis instead of numerical simulation.  If you need to refresh your memory, re-read the Problem Setting from Section 1.  While everything can be done with only pen and paper, we will use Python to automate some calculating and plotting.

## Formulas

In Section 1, Exercise 1.3, you came up with formulas for how you did for the day:

$$\begin{aligned}
    \text{sales} &= \text{smaller of order quantity and demand} \\
    \text{revenue} &= \text{retail price} \times \text{sales} \\
    \text{cost} &= \text{wholesale cost} \times \text{order quantity} \\
    \text{profit} &= \text{revenue} - \text{cost}
  \end{aligned} $$
  
We can write this more concisely using our variables $p$, $c$, $y$, and $D$:
$$\begin{aligned}
    \text{sales} &= \min\{y, D\} \\
    \text{revenue} &= p \min\{y, D\} \\
    \text{cost} &= c y \\
    \text{profit} &= p \min\{y, D\} - c y
  \end{aligned} $$

In case you are unfamiliar with this notation, $\min\{x, y\}$ is a function that gives the smaller (or minimum) of its two arguments.
$$\min\{x, y\} \overset{\text{def}}{=} \begin{cases}
    x, &\text{if } x \lt y \\
    y, &\text{if } x \geq y
  \end{cases}$$
   
An analogous function, $\max\{x, y\}$, gives the larger (or maximum) of its two arguments.
$$\max\{x, y\} \overset{\text{def}}{=} \begin{cases}
    y, &\text{if } x \lt y \\
    x, &\text{if } x \geq y
  \end{cases}$$

## Setup

The following Python code imports the necessary Python packages and specifies the problem parameters for the remainder of this notebook.  You can change the parameters if you'd like and all the code should still work, but their output may be mismatched with the written text.

In [1]:
# Import Python packages
import numpy as np

import newsvendor
import news_plot

news = newsvendor.Newsvendor()

Loading BokehJS ...

## Visualizing Relationships

Since we have algebraic formulas for sales, revenue, cost, and profit, we can plot them as a function of demand.  This is analogous to the scatter plots made from simulated data in Section 1.

In [2]:
fields = ['sales', 'revenue', 'cost', 'profit']
labels = {field: {'x': 'demand', 'y': field, 'title': field} for field in fields}
line_func = lambda n: news_plot.line_glyphs(n.calc_data(np.arange(news.min_order, news.max_order+1)), 'demand')
newsvendor.interact_gridplot(news_plot.figures(fields, plot_width=350, plot_height=350), line_func, news, ncols=2, labels=labels)

interactive(children=(IntSlider(value=15, description='Order Qty', max=19), Output()), _dom_classes=('widget-i…

**Remark:** You may feel that this is misleading because demand is discrete, while the visualization presented implies a continuous relationship.  You are correct, but we will see in later sections that these relationships generalize to continuous demand.  If you prefer, you can add an argument `'.'` to each `plot` command, e.g., `plot(demand, sales, '.')`

### Visualizing Probability Distributions

Since demand is random, any quantity that depends on the demand will also be random:

* $\text{sales} = \min\{y, D\}$
* $\text{revenue} = p \min\{y, D\}$
* $\text{profit} = p \min\{y, D\} - c y$

We can think of each of these quantities as a *function* of the random variable $D$.

Each of these quantities will also have a probability distribution that are derived from the demand distribution, but will not be the same.  For example,
$$ \mathsf{P}\left(\text{sales} = i\right) = \begin{cases}
      \frac{1}{20} &\text{for } i = 0,\dots,y-1 \\
      1 - \frac{y}{20} &\text{for } i = y \\
   \end{cases} $$
Rephrasing slightly:
$$\begin{aligned}
    \mathsf{P}(\text{sales} = i) &= \mathsf{P}(D = i) \qquad \text{i < y} \\
    \mathsf{P}(\text{sales} = y) &= \mathsf{P}(D \geq y)
  \end{aligned}$$

For the order quantity you specified above, the probability mass function (pmf) of the $D$ and $\min\{y, D\}$ are calculated and shown below.  These are analogous to the histograms made from the simulated data in Section 1.

In [3]:
# Bar plots of probability mass functions with order quantity slider (compare to histograms from Section 1)
fields = ['demand', 'sales', 'profit']
labels = {field: {'x': field, 'y': 'probability', 'title': field + ' pmf'} for field in fields}
pmf_func = lambda n: news_plot.bar_glyphs(n.pmfs())
newsvendor.interact_gridplot(news_plot.figures(fields), pmf_func, news, labels=labels)

interactive(children=(IntSlider(value=15, description='Order Qty', max=19), Output()), _dom_classes=('widget-i…

### Calculating Expected Values

Each of these random quantities also has an expected value -- a probability-weighted average value.  Recall from your introductory probability class, for a discrete random variable $X$ with support $\mathcal{S}$ (the set of possible values $X$ can take), its expected value is defined to be

$$ \mathsf{E}[X] = \sum_{k \in \mathcal{S}} k \mathsf{P}(X = k) $$

Remember that, $\mathsf{E}[\text{sales}] = \mathsf{E}[\min\{y, D\}] \neq \min\{y, \mathsf{E}[D]\}$, but $\mathsf{E}[\text{profit}] = \mathsf{E}[p \min\{y, D\} - c y] = p \mathsf{E}[\min\{y, D\}] - c y$.  This is because profit is a linear function of sales, but sales is a non-linear function of demand.

This is equivalent to the part in Section 1 (“Taking a Longer-Term Perspective”) where we looked at average daily values over a large set of data.

### Exercise 2.1:

Calculate the expected values of: demand, sales, and profit using the numeric values for problem parameters and order quantity that you specified.

Use the codeblock below to check your answers.

*Hint:* The expected sales should be the only one that is slightly tricky.  There are two ways to think about this and you can quickly see that they are equivalent.

1. Treat sales as a random variable with the probability mass function specified above in “Visualizing Probability Distributions”:

$$ \mathsf{E}[\text{sales}] = \sum_{i=0}^y i \mathsf{P}(\text{sales} = i) = \sum_{i=0}^{y-1} i \mathsf{P}(D = i) + y \mathsf{P}(D \geq y) $$

2. Treat sales as a function of demand:

$$ \mathsf{E}[\text{sales}] = \mathsf{E}[\min\{y, D\}] = \sum_{i=0}^{19} \min\{y, i\} \mathsf{P}(D = i) = \sum_{i=0}^{y-1} i \mathsf{P}(D = i) + \sum_{i=y}^{19} y \mathsf{P}(D = i)$$

In [4]:
# Table of expected values with order quantity slider
fields = ['demand', 'sales', 'profit']
newsvendor.order_qty_interact(lambda y: news.exp_vals([y])[fields].style.hide_index(), news)

interactive(children=(IntSlider(value=15, description='Order Qty', max=19), Output()), _dom_classes=('widget-i…

### Exercise 2.2:

Come up with formulas for the expected values of: sales, revenue, and profit as a function of order quantity $y$.  You should continue to assume that $D \sim \text{Uniform}\{0,\dots,19\}$, but you can represent the retail price and wholesale cost as variables or numbers.

*Hint:* Use the following summation formula:
$$ \sum_{i=1}^n i = 1 + 2 + \dots + n = \frac{n (n + 1)}{2} $$

## Finding the Optimal Order Quantity

We can now use Python to plot the formulas you came up with to see how your expected sales, revenue, cost, and profit change with your order quantity.  We can also search all possible values of order quantity to find the one(s) that result in the highest expected profit.

This is analogous to Exercise 1.4 in Section 1.

In [5]:
fields = ['sales', 'revenue', 'cost', 'profit']
labels = {field: {'x': 'order qty', 'y': 'expected ' + field, 'title': field} for field in fields}
ev_func = lambda n: news_plot.line_glyphs(n.exp_vals(), 'order qty')
ev_sources, ev_plots = newsvendor.news_gridplot(news_plot.figures(fields, plot_width=350, plot_height=350), ev_func, news, labels=labels)
circle_func = lambda n: news_plot.circle_markers(n.exp_vals([n.order_qty]))
newsvendor.interact_gridplot({field: ev_plots[field] for field in fields}, circle_func, news, ncols=2)

interactive(children=(IntSlider(value=15, description='Order Qty', max=19), Output()), _dom_classes=('widget-i…

**Remark:** Again, you may feel that this is misleading since demand is discrete.  However, we never explicitly required that the order quantity $y$ had to be discrete.  We did *implicitly* assume in our revenue formula that, if $y$ is not an integer and $D > y$ we could sell partially fill a unit of demand: sell a fraction of a newspaper at the pro-rated price.  This seems unrealistic for this setting, but perhaps not for others.  In later sections we will take advantage of continuous demand and continuous order quantities for further insight.

### Exercise 2.3:

Examine the plots above and confirm that the optimal order quantity does indeed maximize expected profit.

Does it make sense that the expected sales, revenue, and cost are all increasing functions of order quantity?

Why is it that the expected profit is increasing and then decreasing?  Can you think of the optimal order quantity in terms of balancing a tradeoff?  (Obviously, the tradeoff is between ordering too much and ordering too little.  But what are the associated costs of ordering too much versus too little?)

### Exercise 2.4:

Generalize the formulas you derived in Exercise 2.2 to incorporate variables for the wholesale cost $c$, retail price $p$, and maximum demand $d$ (so $D \sim \text{Uniform}\{0, \dots, d\}$).

Can you derive a formula for the optimal order quantity (as a function of $p$, $c$, and $d$)?

*Hint:* Your formula for expected profit as a function of $y$ should be a concave quadratic function.  You can find the maximum of this function by solving for where its derivative equals 0.